In [1]:
setwd("D:/OneDrive - g.ntu.edu.tw/110/流病與統計")
resp <- read.csv("resp.csv",header=TRUE)
attach(resp)
head(resp)

center,id,treatment,gender,age,baseline,visit,outcome
1,1,P,M,46,0,1,0
1,1,P,M,46,0,2,0
1,1,P,M,46,0,3,0
1,1,P,M,46,0,4,0
1,2,P,M,28,0,1,0
1,2,P,M,28,0,2,0


In [2]:
newtonraphson <- function(ftn, x0, tol = 1e-9, max.iter = 100) {
 x <- x0 # x0: the initial value
 fx <- ftn(x)
 iter <- 0
 while ((max(abs(fx[[1]])) > tol) & (iter < max.iter)) {
 x <- x - solve(fx[[2]]) %*% fx[[1]]
 fx <- ftn(x)
iter <- iter + 1
 }
 if (max(abs(fx[[1]])) > tol) {
 cat('Algorithm failed to converge\n')
return(NULL)
 } else { # max(abs(fx[[1]])) <= tol
cat("Algorithm converged\n")
return(x)
 }
}


In [3]:
ftn <- function(beta_coef){
    mat <- cbind(rep(1,length(treatment)),ifelse(treatment=="P",1,0),age,baseline)
    p <- exp(mat%*%beta_coef)/(1+exp(mat%*%beta_coef))
    grad <- t(mat)%*%(outcome-p)
    hess <- -t(mat)%*%diag(c(p*(1-p)),length(outcome))%*%mat
    return(list(grad,hess))
}

In [10]:
typeof(ftn(c(0,0,0,0))[[1]])

[1] "double"

In [6]:
beta <- newtonraphson(ftn,c(0,0,0,0))
cat("\n\t\tbeta\n
intercept:   ",beta[1],"\n
treatment:  ",beta[2],"\n
AGE:\t    ",beta[3],"\n
baseline:    ",beta[4])

Algorithm converged

		beta

intercept:    0.4367055 

treatment:   -1.234759 

AGE:	     -0.01140389 

baseline:     1.982412

In [5]:
solve(-ftn(beta)[[2]])

,,,age,baseline
,0.09951118,-2.070235e-02,-2.168930e-03,-1.289550e-02
,-0.02070235,5.133126e-02,-3.292548e-05,-1.351023e-02
age,-0.00216893,-3.292548e-05,6.630252e-05,-2.084783e-05
baseline,-0.01289550,-1.351023e-02,-2.084783e-05,5.426992e-02


In [6]:
logL <- function(beta){
    mat <- cbind(rep(1,length(treatment)),ifelse(treatment=="P",1,0),age,baseline)
    logli <- 0
    for(i in 1:length(outcome)){
        logli <- logli+(outcome[i]*mat[i,]%*%beta-log(1+exp(mat[i,]%*%beta)))
    }
    return(logli)
} 

In [7]:
cat("log likelihood at beta: ",logL(beta))

log likelihood at beta:  -247.9434